In [2]:
!pip install -U scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 11.3 MB/s eta 0:00:00


In [20]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Definir las variables difusas de entrada (hora del día y ocupación) y la salida (intensidad de la luz)
time_of_day = ctrl.Antecedent(np.arange(0, 24, 1), 'time_of_day')  # Rango de 0 a 24 horas
occupancy = ctrl.Antecedent(np.arange(0, 2, 1), 'occupancy')  # 0: Desocupado, 1: Ocupado
lighting = ctrl.Consequent(np.arange(0, 101, 1), 'lighting')  # Intensidad de luz de 0 a 100%

# conjuntos difusos para 'time_of_day'
time_of_day['morning'] = fuzz.trimf(time_of_day.universe, [6, 8, 12])# Mañana: 6 AM - 12 PM
time_of_day['afternoon'] = fuzz.trimf(time_of_day.universe, [12, 15, 18])# Tarde: 12 PM - 6 PM
time_of_day['evening'] = fuzz.trimf(time_of_day.universe, [18, 20, 21])# Tarde-noche: 6 PM - 9 PM
time_of_day['night'] = fuzz.trimf(time_of_day.universe, [21, 24, 30])
# Noche: 9 PM - 6 AM (se extiende a 30 para manejar medianoche)

# Definir los conjuntos difusos para 'occupancy'
occupancy['unoccupied'] = fuzz.trimf(occupancy.universe, [0, 0, 1])
occupancy['occupied'] = fuzz.trimf(occupancy.universe, [0, 1, 1])

# Definir los conjuntos difusos para 'lighting'
lighting['off'] = fuzz.trimf(lighting.universe, [0, 0, 25])
lighting['low'] = fuzz.trimf(lighting.universe, [0, 25, 50])
lighting['medium'] = fuzz.trimf(lighting.universe, [25, 50, 75])
lighting['high'] = fuzz.trimf(lighting.universe, [50, 100, 100])

# Pedir al usuario sus preferencias de iluminación para diferentes momentos del día
print("Por favor, configure sus preferencias de iluminación:")
user_preferences = {
    'morning': int(input("Nivel de iluminación para la mañana (0-100%): ")),
    'afternoon': int(input("Nivel de iluminación para la tarde (0-100%): ")),
    'evening': int(input("Nivel de iluminación para la tarde-noche (0-100%): ")),
    'night': int(input("Nivel de iluminación para la noche (0-100%): "))
}

# Ajustar reglas en base a las preferencias del usuario
if user_preferences['morning'] == 0:
    rule1 = ctrl.Rule(time_of_day['morning'] & occupancy['occupied'], lighting['off'])
elif user_preferences['morning'] <= 25:
    rule1 = ctrl.Rule(time_of_day['morning'] & occupancy['occupied'], lighting['off'])
elif user_preferences['morning'] <= 50:
    rule1 = ctrl.Rule(time_of_day['morning'] & occupancy['occupied'], lighting['low'])
elif user_preferences['morning'] <= 75:
    rule1 = ctrl.Rule(time_of_day['morning'] & occupancy['occupied'], lighting['medium'])
else:
    rule1 = ctrl.Rule(time_of_day['morning'] & occupancy['occupied'], lighting['high'])

if user_preferences['afternoon'] == 0:
    rule2 = ctrl.Rule(time_of_day['afternoon'] & occupancy['occupied'], lighting['off'])
elif user_preferences['afternoon'] <= 25:
    rule2 = ctrl.Rule(time_of_day['afternoon'] & occupancy['occupied'], lighting['off'])
elif user_preferences['afternoon'] <= 50:
    rule2 = ctrl.Rule(time_of_day['afternoon'] & occupancy['occupied'], lighting['low'])
elif user_preferences['afternoon'] <= 75:
    rule2 = ctrl.Rule(time_of_day['afternoon'] & occupancy['occupied'], lighting['medium'])
else:
    rule2 = ctrl.Rule(time_of_day['afternoon'] & occupancy['occupied'], lighting['high'])

if user_preferences['evening'] == 0:
    rule3 = ctrl.Rule(time_of_day['evening'] & occupancy['occupied'], lighting['off'])
elif user_preferences['evening'] <= 25:
    rule3 = ctrl.Rule(time_of_day['evening'] & occupancy['occupied'], lighting['off'])
elif user_preferences['evening'] <= 50:
    rule3 = ctrl.Rule(time_of_day['evening'] & occupancy['occupied'], lighting['low'])
elif user_preferences['evening'] <= 75:
    rule3 = ctrl.Rule(time_of_day['evening'] & occupancy['occupied'], lighting['medium'])
else:
    rule3 = ctrl.Rule(time_of_day['evening'] & occupancy['occupied'], lighting['high'])

if user_preferences['night'] == 0:
    rule4 = ctrl.Rule(time_of_day['night'] & occupancy['unoccupied'], lighting['off'])
elif user_preferences['night'] <= 25:
    rule4 = ctrl.Rule(time_of_day['night'] & occupancy['unoccupied'], lighting['off'])
elif user_preferences['night'] <= 50:
    rule4 = ctrl.Rule(time_of_day['night'] & occupancy['unoccupied'], lighting['low'])
elif user_preferences['night'] <= 75:
    rule4 = ctrl.Rule(time_of_day['night'] & occupancy['unoccupied'], lighting['medium'])
else:
    rule4 = ctrl.Rule(time_of_day['night'] & occupancy['unoccupied'], lighting['high'])




# Definir reglas adicionales para cuando la ocupación es 'unoccupied' (desocupada)
rule5 = ctrl.Rule(time_of_day['morning'] & occupancy['unoccupied'], lighting['off'])
rule6 = ctrl.Rule(time_of_day['afternoon'] & occupancy['unoccupied'], lighting['off'])
rule7 = ctrl.Rule(time_of_day['evening'] & occupancy['unoccupied'], lighting['off'])
rule8 = ctrl.Rule(time_of_day['night'] & occupancy['unoccupied'], lighting['off'])

# Crear el sistema de control difuso con todas las reglas
lighting_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8])
lighting_simulation = ctrl.ControlSystemSimulation(lighting_ctrl)

# se definen ejemplos de entrada (hora del dia y si la habitacion esta ocupada, 1 esta ocupada, 0 desocupada)
example_data = [
    {'time': 7, 'occupancy': 1},  # Mañana ocupada
    {'time': 13, 'occupancy': 0},  # Tarde desocupada
    {'time': 19, 'occupancy': 1},  # Tarde-noche ocupada
    {'time': 23, 'occupancy': 0},  # Noche desocupada
]

# Simular las condiciones para los ejemplos
for example in example_data:
    lighting_simulation.input['time_of_day'] = example['time']
    lighting_simulation.input['occupancy'] = example['occupancy']

    # Calcular la intensidad de la luz
    lighting_simulation.compute()
    output_lighting = lighting_simulation.output.get('lighting', None)

    # Verificar si se genera una salida y mostrar el resultado
    if output_lighting is not None:
        print(f"Para la hora {example['time']} y ocupación {example['occupancy']}: La intensidad de la luz es {output_lighting:.2f}%")
    else:
        print(f"Para la hora {example['time']} y ocupación {example['occupancy']}: No se pudo generar una salida de iluminación")


Por favor, configure sus preferencias de iluminación:
Nivel de iluminación para la mañana (0-100%): 5
Nivel de iluminación para la tarde (0-100%): 20
Nivel de iluminación para la tarde-noche (0-100%): 50
Nivel de iluminación para la noche (0-100%): 2
Para la hora 7 y ocupación 1: La intensidad de la luz es 9.72%
Para la hora 13 y ocupación 0: La intensidad de la luz es 10.56%
Para la hora 19 y ocupación 1: La intensidad de la luz es 25.00%
Para la hora 23 y ocupación 0: La intensidad de la luz es 9.03%
